# 人脸识别我也行

硬件准备：摄像头、小音箱

注意事项：
- 文件夹目录中"unknown.jpg"为待测图像，"face_photo"文件夹中为来源网络的素材图片,"xuguai.py"为本教材封装库。
- 请逐步运行单元格中的代码，即可查看具体效果。
- 请确认硬件处于上网状态，否则无法使用百度API相关功能。

### 准备工作:加载第三方库

教材专用库提前封装，将百度ai平台的人脸识别接口封装成baidu_face函数，去掉了数据处理的部分

In [1]:
# -*- coding: utf-8 -*-
import face_recognition as fr
import cv2
import os
import base64
from aip import AipFace
from xuguai import baidu_face


### 基础知识:CV2图像处理相关的一些基本函数

In [ ]:
import cv2
#读取图像，斜杠在Python的字符串中有转义作用，在路径前方加r的作用是保持字符串原始值的意思,这里的图片使用了绝对路径。
img = cv2.imread(r"E:\photo\1.jpg")  
#拍照，使用基于opencv库已经封装好的拍照函数
getpic("test.jpg")
#在图像上添加矩形,这里的四个变量代表矩形四个顶点的位置
top,right,bottom,left=(121, 162, 196, 88)
cv2.rectangle(img,(left, top),(right, bottom),color = (50,255,50),thickness = 3)
#在图像上写字
img = cv2.putText(img, '000', (x=100,y=100), font = cv2.FONT_HERSHEY_SIMPLEX , 1.2, (255, 255, 255), 2)

### 应用实例1：百度API人脸识别

In [3]:


APP_ID = '15377544'
API_KEY = '0qK3r7t8i5SR9ir9wmUsXWMQ'
SECRET_KEY = 'hhbbQzWhRF0ejZFeIZ95aGsrZFirlGgP'
aipFace = AipFace(APP_ID, API_KEY, SECRET_KEY)  # 填写参数，初始化AipFace对象
#调用人脸识别函数读取图片进行识别
result=baidu_face("unknown.jpg",aipFace)
print("百度AI平台识别返回结果：")
#观察整体的返回值，face_num人脸数量，我们选取其中第一个人的数据，得出被测试者的评分
print(str(result).replace(',','\n'))
firstman=result['result']['face_list'][0]     
print('预计年龄:',firstman['age'])

百度AI平台识别返回结果：
{'error_code': 0
 'error_msg': 'SUCCESS'
 'log_id': 4525897999840
 'timestamp': 1586837116
 'cached': 0
 'result': {'face_num': 1
 'face_list': [{'face_token': 'f775a1735f84735d96b612a6365816fd'
 'location': {'left': 159.47
 'top': 87.6
 'width': 94
 'height': 89
 'rotation': -2}
 'face_probability': 1
 'angle': {'yaw': 26.71
 'pitch': 12.58
 'roll': -6.82}
 'age': 25
 'expression': {'type': 'smile'
 'probability': 0.99}
 'beauty': 70.25}]}}
预计年龄: 25


### 应用实例2：face_recognition库中检测人脸函数face_locations

In [ ]:
# 读取图片并识别人脸坐标
img = fr.load_image_file("face_photo/meeting.jpg")
face_locations = fr.face_locations(img)
#调用opencv函数加载原图片
img = cv2.imread("face_photo/meeting.jpg")
# 遍历每个人脸，并标注
faceNum = len(face_locations)
for i in range(0, faceNum):
    top,right,bottom,left=(face_locations[i][0],face_locations[i][1],face_locations[i][2], face_locations[i][3])
    cv2.rectangle(img, (left, top),(right, bottom),color = (50,255,50),thickness = 3)
# 显示识别结果
cv2.imshow("recognition", img)
# waitkey函数为窗口持续时间，单位为毫秒，参数0为永久保持窗口
cv2.waitKey(0)


### 应用实例3：人脸比较

In [6]:
#加载人脸照片
liuxiang_image = fr.load_image_file("face_photo/subingtian.jpg");
zhangjike_image = fr.load_image_file("face_photo/zhangjike.jpg");
yaoming_image = fr.load_image_file("face_photo/yaoming.jpg");
unknown_image = fr.load_image_file("unknown.jpg");
#将图片数据转化为向量
liuxiang_encoding = fr.face_encodings(liuxiang_image)[0]
zhangjike_encoding = fr.face_encodings(zhangjike_image)[0]
yaoming_encoding = fr.face_encodings(yaoming_image)[0]
unknown_encoding = fr.face_encodings(unknown_image)[0]
#比较函数face_recognition.compare_faces
results = fr.compare_faces([liuxiang_encoding, zhangjike_encoding, yaoming_encoding], unknown_encoding )
labels = ['liuxiang', 'zhangjike','yaoming']
for i in range(0, len(results)):
    print(labels[i]+" "+str(results[i]))
#输出识别结果为True的照片所对应的标签
for i in range(0, len(results)):
    if results[i] == True:
        print('The person is:'+labels[i])

liuxiang False
zhangjike False
yaoming True
The person is:yaoming


# 实践活动1：会“数人脸”的机器人

实践内容：识别当前摄像头前的人脸数量，并且通过语音合成功能进行播报。

### 实践步骤1：实时录像

In [3]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    cv2.imshow('Video', frame)
    #按q退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()


### 实践步骤2：核心逻辑

调用摄像头进行拍摄，对照片进行人脸识别操作，当照片中出现人脸时，播报人脸数量。

In [10]:
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    # 读取图片并识别人脸坐标
    face_locations = fr.face_locations(frame)
    # 遍历每个人脸，并标注
    faceNum = len(face_locations)
    for i in range(0, faceNum):
        top,right,bottom,left=(face_locations[i][0],face_locations[i][1],face_locations[i][2], face_locations[i][3])
        cv2.rectangle(frame, (left, top),(right, bottom),color = (50,255,50),thickness = 3)
    frame = cv2.putText(frame, 'facenum:'+str(faceNum), (100,100), cv2.FONT_HERSHEY_SIMPLEX,1.2, (255, 255, 255), 2)
    cv2.imshow('Video', frame)
    #按q退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

### 实践步骤3：功能优化

* 想要完成播报的效果，不一定要重复调用语音合成功能。比如常见的收款播报提醒，成功支付后播放：“账户收款：321元”。前面固定部分的音频可以预先合成，而数字的合成也只要录制好数字语音和金钱单位。使用时，根据用户输入组合输出即可。
* 完善系统，将人脸框出，并标注数字等信息。

# 实践活动2：校园智能门禁系统

实践内容：制作一个能够批量导入小组成员的人脸信息，认出特定人脸的名称，识别出对应的姓名的校园智能门禁系统。

### 实践步骤1：数据收集，拍照时应保证光线充足，使拍出来的照片清晰可识别

用基本的CV2拍照代码，将小组成员照片存入本地文件夹中，注意文件路径是否存在。

In [ ]:
#拍照，使用基于opencv库已经封装好的拍照函数
getpic("team_photo\name.jpg")  

### 实践步骤2：批量读取文件夹内的照片

通过比较函数face_recognition.compare_faces与"unknown.jpg"进行对比

In [ ]:
num=0
a=[]
#facecompare一次可以读取最多二十张照片进行比较
def readname():
    filePath = "face_photo"
    name = os.listdir(filePath)
    return name
 
#创建一个装有姓名的列表
name = readname()
print(name)
for i in name:
    num=num+1
    img=fr.load_image_file("face_photo/" + i )
    try:
        img=fr.face_encodings(img)[0]
    except:#如果无法从图片中识别到人脸会报错
        pass
    a.insert(num,img)
    
#未知图片
unknown_image = fr.load_image_file("unknown.jpg");
unknown_encoding = fr.face_encodings(unknown_image)[0]

#比较函数face_recognition.compare_faces
results = fr.compare_faces( a , unknown_encoding )
labels = name

#输出结果
print('results:'+str(results))
for i in range(0, len(results)):
    if results[i] == True:
        print('The person is:'+labels[i])


### 实践步骤3：功能优化

识别出特定人员后，作出反馈，比如指示灯亮起，播报欢迎提示等。

### 实践总结

1.

2.
